In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import pylab
from plotly.subplots import make_subplots
from tqdm import tqdm
%matplotlib auto
%config InlineBackend.figure_format = 'retina'
import random
import time
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
from numpy import linalg 
from itertools import accumulate
import cvxpy as cp
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
import scipy.io as scio
import numpy as np
import pandas as pd
import torch
from cvxpylayers.torch import CvxpyLayer
import plotly.graph_objects as go

amb_data = np.array(pd.read_excel('input_data_pool.xlsx',sheet_name='theta_amb')['theta_amb'])
price_data = np.array(pd.read_excel('input_data_pool.xlsx',sheet_name='price')['price'])

Using matplotlib backend: MacOSX


In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self, d0=24*2, d1 = 24):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(d0, d1)
        self.fc2 = nn.Linear(d1, d1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=48, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=24, bias=True)
)


In [68]:
input_price = torch.tensor(price_data[:24*5]).reshape(5,24)
input_amb = torch.tensor(amb_data[:24*5]).reshape(5,24)
input_tensor = torch.concat([input_price,input_amb],dim=1).to(torch.float32)
output_tensor = true[0].reshape(5,24).to(torch.float32)

def train(iters):
    layer = Net()
    
    results = []
    show = True
    optimizer = torch.optim.Adam(layer.parameters(), lr=0.1)
    for i in range(iters):
        torch.manual_seed(1)
        np.random.seed(1)
               
        pred = layer(input_tensor)        
        loss = nn.MSELoss()(output_tensor ,pred)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        results.append(loss.item())
        if i % 100==0: print("(iter %d) loss: %g " % (i, results[-1]))
        #if i % 100 == 1: print(variables)
        if i == 50:
            optimizer.param_groups[0]["lr"] = 0.05
        if show:
            im = plt.plot(results,color='gray')
            anno = plt.annotate(f'step:{i}\n loss={loss}', xy=(0.85, 0.9), xycoords='axes fraction',color='black')
            plt.axis("equal")
            plt.pause(0.001)
            anno.remove()
        
    return layer


In [69]:
model = train(500)

(iter 0) loss: 12.1344 
(iter 100) loss: 0.239375 
(iter 200) loss: 0.23935 
(iter 300) loss: 0.23935 
(iter 400) loss: 0.23935 


In [73]:
test = torch.concat([torch.tensor(price_data[T:T+24]), torch.tensor(amb_data[T:T+24])]).to(torch.float32)
model(test).detach().numpy()

array([ 6.6942090e-01,  1.6431214e-01,  2.0906754e-01,  4.9845584e-02,
        1.5641318e-01,  7.1646237e-01,  1.0169713e-12,  9.9184968e-02,
       -1.9547389e-12,  1.9659850e-01,  5.1256686e-01,  1.2248969e+00,
        1.1984836e+00,  1.2354234e+00,  1.4452894e+00,  1.9070957e+00,
        1.9169872e+00,  1.7665367e+00,  1.5991712e+00,  1.2514955e+00,
        1.5215962e-01, -8.4275821e-13, -2.5093074e-12,  1.1292929e-02],
      dtype=float32)

In [63]:
T = 24*5
price = price_data[:T]
amb = amb_data[:T]
variable = {'pn': 0.02, 'a1': 0.9521839137854551, 'a2': 5.598830495762315, 'a3': 0.2677141616859941, 'max_theta': 20.539969677000002, 'min_theta': 18.670106323, 'max_power': 5.38}
pn_value = variable['pn']
a1_value = variable['a1']
a2_value = variable['a2']
max_theta = variable['max_theta']
min_theta = variable['min_theta']
max_power = variable['max_power']
theta_0 = 21.64671372
true = solve(price, amb, T, pn_value, a1_value, a2_value, max_theta, min_theta, max_power,theta_0, tensor=True)

In [4]:
def solve(price, amb, T, pn, a1, a2, max_theta, min_theta, max_power, theta_0, tensor=False):
    p = cp.Variable(T)
    y = cp.Variable(T)
    theta = cp.Variable(T)
    obj = price@p + pn * cp.sum_squares(y)
    cons = [y >= 0, p >= 0]
    for i in range(T):
        if i == 0:
            cons.append(theta[i] == a1*theta_0 + (1-a1)*(amb[i] - a2*p[i]))
        else:
            cons.append(theta[i] == a1*theta[i-1] + (1-a1)*(amb[i] - a2*p[i]))
        cons.append(p[i] <= max_power)
        cons.append(theta[i] <= max_theta + y[i])
        cons.append(theta[i] >= min_theta - y[i])
    problem = cp.Problem(cp.Minimize(obj), cons)
    problem.solve()
    if tensor:
        return torch.tensor(p.value),torch.tensor(theta.value),torch.tensor(y.value)
    return p.value, theta.value, y.value